In [2]:
import numpy as np
import pandas as pd
import chardet
import re

In [3]:
#Se define la ruta de la información.
data_propiedades = "../Data/properati.csv"

In [4]:
#Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

In [26]:
#Chequeamos que los datos se hayan importado correctamente
data.head(3)

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


<strong>Renombro la columna Unnamed: 0 como Index, podría ser codigo de propiedad.</strong>

In [6]:
data.rename(columns={'Unnamed: 0': "Index"}, inplace=True)
data.head(3)

,Index,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [7]:
data.dtypes

Index                           int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [35]:
# Armo un nuevo DataFrame con los valores que pueden resultar de interes descarto, los datos de posicion geografica.
data_precio = pd.DataFrame(data, columns=['Index','country_name','state_name','place_name', 'geonames_id','price','currency', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2'])
data_precio.head()

,Index,country_name,state_name,place_name,geonames_id,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2
0,0,Argentina,Capital Federal,Mataderos,3430787.0,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000
1,1,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,3432039.0,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN
2,2,Argentina,Capital Federal,Mataderos,3430787.0,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909
3,3,Argentina,Capital Federal,Liniers,3431333.0,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN
4,4,Argentina,Buenos Aires Costa Atlántica,Centro,3435548.0,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429


Voy a realizar una analisis diferente, primero reviso el pais el municipio y el lugar.

In [32]:
# Reviso si tengo algun otro pais.
data_precio.country_name.value_counts()

Argentina    121220
Name: country_name, dtype: int64

In [33]:
data_precio.state_name.value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [34]:
data_precio.place_name.value_counts()

Córdoba                           9254
Rosario                           8504
Mar del Plata                     6534
Tigre                             3324
Nordelta                          3315
                                  ... 
Barrio Cerrado "La Candelaria"       1
Country Golf El Sosiego              1
Obispo Trejo                         1
Villa Paranacito                     1
Vistalba                             1
Name: place_name, Length: 1060, dtype: int64

Vuelvo a calcular el tipo de cambio

In [42]:

valores['usdtolocal'] = valores.price_aprox_local_currency / valores.price_aprox_usd
cambio_usd_max = valores.usdtolocal.max()
cambio_usd_min = valores.usdtolocal.min()

# Verifico si en el calculo el valor max y min son iguales.
print(round(cambio_usd_max, 2))
print(round(cambio_usd_min, 2))

17.64
17.64


In [45]:
check_change = round(valores.usdtolocal, 2) == round(cambio_usd_max, 2)

print(f'Son todos igual a {round(cambio_usd_max, 2)}: {all(check_change)}')

print(f'Hay valores que son NaN?: {any(valores.usdtolocal.isnull())}')

Son todos igual a 17.64: False
Hay valores que son NaN?: True


Revisando tenemos <u>Price</u> y <u>Price_aprox_usd</u> que parecen ser lo mismo ¿? vamos a tratar de compararlos.

In [48]:
# revisar price contra price_aprox_usd
check_prices_usd = valores.price == valores.price_aprox_usd
print(f'El valor de price es igual a price_aprox_usd?: {all(check_prices_usd)}')
print(f'Hay precios con campos NaN? : {any(valores.price.isnull())}') # Hay precios que son Nulos.

El valor de price es igual a price_aprox_usd?: False
Hay precios con campos NaN? : True


Con esto detectamos que el precio y precio aproximado parecen no ser lo mismo tampoco.

In [52]:
valores['price_usd']= round(valores.apply(lambda x: x['price']  if x['currency'] == 'USD' else (x['price'] * cambio_usd if x['currency'] == 'ARS' else np.NaN), axis=1),2)
valores.sample(10)

,Index,country_name,state_name,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,usdtolocal,price_usd
91374,91374,Argentina,Córdoba,Córdoba,1093000.0,ARS,1081598.14,61299.45,60.0,48.0,1021.657500,22770.833333,17.6445,19285438.5
10232,10232,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,63000.0,USD,1111603.50,63000.00,36.0,36.0,1750.000000,1750.000000,17.6445,63000.0
46748,46748,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,94900.0,USD,1674463.05,94900.00,149.0,NaN,636.912752,NaN,17.6445,94900.0
109348,109348,Argentina,Santa Fe,Rosario,NaN,NaN,NaN,NaN,140.0,105.0,NaN,NaN,NaN,NaN
364,364,Argentina,Capital Federal,Paternal,1348200.0,ARS,1334136.11,75612.01,33.0,33.0,2291.273030,40854.545455,17.6445,23788314.9
7776,7776,Argentina,Entre Ríos,Paraná,NaN,NaN,NaN,NaN,40.0,40.0,NaN,NaN,NaN,NaN
15563,15563,Argentina,Capital Federal,Flores,275000.0,USD,4852237.50,275000.00,108.0,107.0,2546.296296,2570.093458,17.6445,275000.0
78682,78682,Argentina,Capital Federal,Flores,112000.0,USD,1976184.00,112000.00,43.0,39.0,2604.651163,2871.794872,17.6445,112000.0
83291,83291,Argentina,Córdoba,Córdoba,1750000.0,ARS,1731744.68,98146.43,NaN,42.0,NaN,41666.666667,17.6445,30877875.0
70860,70860,Argentina,Bs.As. G.B.A. Zona Sur,Banfield,170000.0,USD,2999565.00,170000.00,196.0,NaN,867.346939,NaN,17.6445,170000.0


<h2>$ Hay que definir si conservamos todos los valores en dolares, o completamos los campos con valores en Pesos.</h2>

Ejemplo, vamos a traer el dato con indice: 46748

In [58]:
valores.iloc[46748]

Index                                                46748
country_name                                     Argentina
state_name                    Buenos Aires Costa Atlántica
place_name                                   Mar del Plata
price                                                94900
currency                                               USD
price_aprox_local_currency                     1.67446e+06
price_aprox_usd                                      94900
surface_total_in_m2                                    149
surface_covered_in_m2                                  NaN
price_usd_per_m2                                   636.913
price_per_m2                                           NaN
usdtolocal                                         17.6445
price_usd                                            94900
Name: 46748, dtype: object

Se puede observar que falta completar el <strong>price_per_m2</strong> en la moneda local. Pero!! se puede calcular en dolares la superficie cubierta porque tenemos el precio en USD por m2, <strong>dividiendo el valor por el precio por metro cuadrado, obtendriamos el dato surface_covered_in_m2</strong>

<h3>Analizo los datos a través del tipo de moneda.</h3>

In [86]:
valores.currency.value_counts()

USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64

In [94]:
# Creo una mask para traer esos valores.
currency_PEN_mask = valores.currency == 'PEN'
valores.loc[currency_PEN_mask, :]

,Index,country_name,state_name,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,usdtolocal,price_usd
50387,50387,Argentina,Capital Federal,Capital Federal,380000.0,PEN,2066864.90,117139.33,NaN,NaN,NaN,NaN,17.6445,NaN
50388,50388,Argentina,Capital Federal,Capital Federal,950000.0,PEN,5167162.35,292848.33,NaN,NaN,NaN,NaN,17.6445,NaN


In [106]:
# Creo una mask para traer esos valores.
currency_UYU_mask = valores.currency == 'UYU'
valores.loc[currency_UYU_mask, :]

,Index,country_name,state_name,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,usdtolocal,price_usd
107390,107390,Argentina,Mendoza,San Roque,650000000.0,UYU,4.054773e+08,22980378.29,NaN,NaN,NaN,NaN,17.6445,36838674.94


De estas dos ultimas tablas se analiza que en realidad hay que renombrar la moneda, que es USD en ambos casos y tambien me doy cuenta que el calculo que hicimos de price_usd no se completo para estos casos por el tipo de moneda.

In [107]:
valores['price_usd']= round(valores.apply(lambda x: x['price']  if x['currency'] == 'USD' else (x['price'] / cambio_usd if x['currency'] == 'ARS' else x['price_aprox_usd']), axis=1),2)


In [108]:
currency_PEN_mask = valores.currency == 'PEN'
valores.loc[currency_PEN_mask, :]

,Index,country_name,state_name,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,usdtolocal,price_usd
50387,50387,Argentina,Capital Federal,Capital Federal,380000.0,PEN,2066864.90,117139.33,NaN,NaN,NaN,NaN,17.6445,117139.33
50388,50388,Argentina,Capital Federal,Capital Federal,950000.0,PEN,5167162.35,292848.33,NaN,NaN,NaN,NaN,17.6445,292848.33


<h3>Vamos a realizar el analisis de los datos de Capital Federal unicamente. Para ver que grado de completitud tengo en los datos, y ver si puedo estimar los datos faltantes.<h/3>

<h1>Capital Federal</h1>

In [109]:
valores_caba_mask = valores.state_name == 'Capital Federal'

valores_caba = valores.loc[valores_caba_mask, :]

valores_caba.sample(10)

,Index,country_name,state_name,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,usdtolocal,price_usd
11385,11385,Argentina,Capital Federal,Nuñez,375000.0,USD,6616687.50,375000.0,NaN,74.0,NaN,5067.567568,17.6445,375000.0
10576,10576,Argentina,Capital Federal,Parque Centenario,160000.0,USD,2823120.00,160000.0,68.0,43.0,2352.941176,3720.930233,17.6445,160000.0
92602,92602,Argentina,Capital Federal,Belgrano,159000.0,USD,2805475.50,159000.0,57.0,55.0,2789.473684,2890.909091,17.6445,159000.0
39572,39572,Argentina,Capital Federal,Centro / Microcentro,268000.0,USD,4728726.00,268000.0,116.0,95.0,2310.344828,2821.052632,17.6445,268000.0
34205,34205,Argentina,Capital Federal,Villa Devoto,435000.0,USD,7675357.50,435000.0,226.0,NaN,1924.778761,NaN,17.6445,435000.0
74297,74297,Argentina,Capital Federal,Colegiales,105000.0,USD,1852672.50,105000.0,42.0,37.0,2500.000000,2837.837838,17.6445,105000.0
106207,106207,Argentina,Capital Federal,Belgrano,241100.0,USD,4254088.95,241100.0,45.0,39.0,5357.777778,6182.051282,17.6445,241100.0
13799,13799,Argentina,Capital Federal,Villa Luro,330000.0,USD,5822685.00,330000.0,NaN,NaN,NaN,NaN,17.6445,330000.0
67691,67691,Argentina,Capital Federal,Flores,90000.0,USD,1588005.00,90000.0,52.0,52.0,1730.769231,1730.769231,17.6445,90000.0
56680,56680,Argentina,Capital Federal,Palermo,159000.0,USD,2805475.50,159000.0,NaN,46.0,NaN,3456.521739,17.6445,159000.0


Vemos la distribución de propiedades que tenemos por Barrio.

In [110]:
valores_caba.place_name.value_counts()

Belgrano           2992
Palermo            2885
Caballito          2273
Villa Urquiza      1629
Recoleta           1547
                   ... 
Velez Sarsfield      39
Villa Real           37
Villa Soldati        15
Villa Riachuelo       6
Catalinas             4
Name: place_name, Length: 62, dtype: int64

In [102]:
valores_caba.count() / valores_caba.shape[0]

Index                         1.000000
country_name                  1.000000
state_name                    1.000000
place_name                    1.000000
price                         0.897945
currency                      0.897945
price_aprox_local_currency    0.897945
price_aprox_usd               0.897945
surface_total_in_m2           0.818078
surface_covered_in_m2         0.927621
price_usd_per_m2              0.725152
price_per_m2                  0.845525
usdtolocal                    0.897945
price_usd                     0.897883
dtype: float64

In [112]:
# Analicemos este registro donde el valor aprox en USD es similar pero no coincide con el valor calculado en usd

valores.iloc[118538]

Index                                  118538
country_name                        Argentina
state_name                    Capital Federal
place_name                          San Telmo
price                              3.4921e+06
currency                                  ARS
price_aprox_local_currency        3.45567e+06
price_aprox_usd                        195850
surface_total_in_m2                        86
surface_covered_in_m2                      67
price_usd_per_m2                      2277.32
price_per_m2                          52120.9
usdtolocal                            17.6445
price_usd                              197914
Name: 118538, dtype: object

In [116]:
# Veamos si el precio calculado me da aproximado el costo de la propiedada.
round(valores.iloc[118538].price_usd_per_m2 * valores.iloc[118538].surface_covered_in_m2, 2)

152580.66